In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import h5py as hdf
from IPython.display import display
from numba import jit
import shutil

In [3]:
def imp(arr):
    plt.figure(figsize=(20,15))
    plt.imshow(arr,cmap='viridis')
    plt.colorbar()
    plt.show()
    #good
def gather_universal(path: str,filetype: str)-> list:
    subdir=path

    data=[]
    for path, subdirs, files in os.walk(subdir):
        for name in files:
            if name.endswith(filetype)  :

                b=(str(os.path.join(path, name)))
                data.append(b)

                #print (str(os.path.join(path, name)))
    return data


In [4]:
#gather hdf5 files
path='Y:\\Istra Repositories\\ak\\LA-G25-01'
files=pd.DataFrame(gather_universal(path,'hdf5'))
files['num']=files[0].apply(lambda x : int(x.split('.')[0].split('_')[-1]))
files.sort_values(by='num',inplace=True)
names=np.array(files[0])
display(names)

array(['Y:\\Istra Repositories\\ak\\LA-G25-01\\series_step_0.hdf5',
       'Y:\\Istra Repositories\\ak\\LA-G25-01\\series_step_1.hdf5',
       'Y:\\Istra Repositories\\ak\\LA-G25-01\\series_step_2.hdf5', ...,
       'Y:\\Istra Repositories\\ak\\LA-G25-01\\series_step_5376.hdf5',
       'Y:\\Istra Repositories\\ak\\LA-G25-01\\series_step_5377.hdf5',
       'Y:\\Istra Repositories\\ak\\LA-G25-01\\series_step_5378.hdf5'],
      dtype=object)

In [7]:
#gather hdf5 datasets
gap=[]
for i in names:
    with hdf.File(i,'r') as r:
        gap.append(np.ndarray.flatten(np.array(r['correlation_load_series_c#00000003']['camera_pos_1'],dtype=np.int32)))
#gap=[np.ndarray.flatten(np.array(hdf.File(i,'r+')['correlation_load_series_camera_1']['camera_pos_1'],dtype=np.int32))for i in names]

data=np.column_stack(gap)

interest=data#[:,40:80]
display(interest.shape)

KeyboardInterrupt: 

In [ ]:
imgsize=(np.int32(1234),np.int32(1624))#interest.iloc[0,:]['data'].shape[0]/1624
display(imgsize)

In [ ]:
before=interest.copy()
imp(before[:,0].reshape(imgsize))
display(before.shape)


In [ ]:
#median anpassen
@jit(nopython=True)
def iron (arr):

    thresh=np.int32(5)
    #plt.plot(arr)
    b=arr.copy()
    med=np.median(arr[:20])
    for i in range(len(b)):

        if (b[i]-med)>thresh:
            b[i]=b[i-1]
    return (b)

#print(len(before))

@jit(nopython=True)
def after_getter(before):
    after=before.copy()
    for i in range(len(after)):
        after[i]=iron(before[i])
    return(after)

after=after_getter(before)


In [ ]:
display(before.shape,after.shape)
def testview(start,end):
    for i in range(start,end):
        display('--------------------------------')
        old=before[:,i].reshape(imgsize)
        imp(old)
        display('++++++++++++++++++++++++++++++++')
        new=after[:,i].reshape(imgsize)
        imp(new)
testview(50,51)


In [ ]:
display('after {}'.format(after.shape))

In [ ]:
def get_attrs(path):

    values=[]
    with hdf.File(path,'r') as f:
        keys=list(f['correlation_load_series_camera_1']['camera_pos_1'].attrs.keys())
        for i in keys:
            values.append(f['correlation_load_series_camera_1']['camera_pos_1'].attrs[i])
    return(keys,values)
#display(get_attrs('data/2020-12-04_Schweißen_2/series_step_55.hdf5'))
def get_analog_attrs(path):

    values=[]
    with hdf.File(path,'r') as f:
        keys=list(f['correlation_load_series_camera_1']['analog_channels'].attrs.keys())
        for i in keys:
            values.append(f['correlation_load_series_camera_1']['analog_channels'].attrs[i])
    return(keys,values)

In [ ]:
exportpath=path+'/export'
os.makedirs(exportpath, exist_ok=True)
for i in names:
    shutil.copy(i, exportpath)

In [ ]:
# sortexportfiles and create right order
exportfiles=pd.DataFrame(gather_universal(exportpath,'hdf5'))
exportfiles['num']=exportfiles[0].apply(lambda x : int(x.split('.')[0].split('_')[-1]))
exportfiles.sort_values(by='num',inplace=True)
exportnames=np.array(exportfiles[0])
display(exportnames)

In [ ]:
# test before export
display(exportnames)
display(after.shape)
with hdf.File(exportnames[50],'r') as r:
    plt.figure(figsize=(20,8))
    plt.imshow(np.array(r['correlation_load_series_camera_1']['camera_pos_1']))
    plt.show()

In [ ]:
#export routine
for i in range(len(after[0,:])):
    res=after[:,i].reshape(1234, 1624).copy()
    with hdf.File(exportnames[i],'r+') as r:
        display(exportnames[i])


        r['correlation_load_series_camera_1']['camera_pos_1'][:,:]=res


In [ ]:
# test after export
with hdf.File(exportnames[50],'r') as r:
    plt.figure(figsize=(20,8))
    plt.imshow(np.array(r['correlation_load_series_camera_1']['camera_pos_1']))
    plt.show()